In [1]:
import numpy as np

import torch
from torch import nn
from transformers import BertPreTrainedModel, BertConfig, BertModel, BertTokenizer, AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification

import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader, RandomSampler, Dataset, SequentialSampler

from load_data_ensemble import initialize_data
from reading_datasets import read_task5
from labels_to_ids import task5_labels_to_ids

import pickle
import json
import csv

In [2]:
class BertEnsemble(BertPreTrainedModel):
  def __init__(self, config, *args, **kwargs):
      super().__init__(config)

      # model 1
      self.bert_model_1 = AutoModelForSequenceClassification.from_pretrained('bert-base-multilingual-uncased', num_labels=3)
      # model 2
      self.bert_model_2 = AutoModelForSequenceClassification.from_pretrained('bert-base-multilingual-uncased', num_labels=3)
      # combine the 2 models into 1
      self.cls = nn.Linear(6, 3)
      self.init_weights()

  def forward(
          self,
          input_ids=None,
          attention_mask=None,
          token_type_ids=None,
          position_ids=None,
          head_mask=None,
          inputs_embeds=None,
          labels=None,
  ):
    outputs = []
    
    input_ids_1 = input_ids[0].to(device, dtype = torch.long)
    attention_mask_1 = attention_mask[0].to(device, dtype = torch.long)
    labels_1 = labels[0].to(device, dtype = torch.long)
    outputs.append(self.bert_model_1(input_ids_1,
                                     attention_mask=attention_mask_1, labels =labels_1))

    input_ids_2 = input_ids[1].to(device, dtype = torch.long)
    attention_mask_2 = attention_mask[1].to(device, dtype = torch.long)
    labels_2 = labels[1].to(device, dtype = torch.long)
    outputs.append(self.bert_model_2(input_ids_2,
                                     attention_mask=attention_mask_2, labels =labels_2))
    print(outputs)

    # just get the [CLS] embeddings
    last_hidden_states = torch.cat([output[1] for output in outputs], dim=1)
    print("Printing last hidden states")
    print(last_hidden_states)
    logits = self.cls(last_hidden_states)
    return logits

In [3]:
device = torch.device("cuda")

config = BertConfig()
model = BertEnsemble(config)
model.to(device)
learning_rate = 1e-05

no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [{
  "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
  }]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

# Training code

In [ ]:
# standard pytorch way of doing things
# 1. create a custom Dataset 
# 2. pass the dataset to a dataloader
# 3. iterate the dataloader and pass the inputs to the model

max_len = 256
batch_size = 6
grad_step = 1
initialization_input = (max_len, batch_size)

model1_tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-uncased', add_prefix_space=True)
model2_tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-uncased', add_prefix_space=True)

#Reading datasets and initializing data loaders
dataset_location = '../2022.07.07_task5/'

#Gives us tweet_id, sentence, and label for each dataset.
train_data = read_task5(dataset_location , split = 'train')
#test_data = read_task5(dataset_location , split = 'dev')#load test set
labels_to_ids = task5_labels_to_ids
#input_data = (train_data, dev_data, labels_to_ids)

dataloader_m1 = initialize_data(model1_tokenizer, initialization_input, train_data, labels_to_ids, shuffle = True)
dataloader_m2 = initialize_data(model2_tokenizer, initialization_input, train_data, labels_to_ids, shuffle = True)

epochs = 5
for epoch in range(epochs):
  # iterate the QA and the AQ inputs simultaneously
  for step, combined_batch in enumerate(zip(dataloader_m1, dataloader_m2)):
    batch_1, batch_2 = combined_batch
    # training so, dropout needed to avoid overfitting
    model.train()
    # move input to GPU
    inputs = {
        "input_ids": [batch_1['input_ids'], batch_2['input_ids']],
        "attention_mask": [batch_1['attention_mask'], batch_2['attention_mask']],
        "labels": [batch_1['labels'], batch_2['labels']]}
    print(type(inputs))
    
    output = model(**inputs)
     
# model outputs are always tuple in transformers (see doc)
    loss = output[0][2]
 
    # backpass
    loss.backward()
    print(f"epoch:{epoch}, loss:{loss}")
    
    # re-calculate the weights
    optimizer.step()
    # again set the grads to 0 for next epoch
    model.zero_grad()
  
  print("\n")

<class 'dict'>
[SequenceClassifierOutput(loss=tensor(1.2588, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[-0.3483,  0.2485,  0.1671],
        [-0.3406, -0.0271, -0.1022],
        [-0.4085,  0.2256, -0.0616],
        [ 0.0439,  0.0538, -0.0476],
        [-0.1109,  0.2911,  0.0754],
        [ 0.0512,  0.0462, -0.2173]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None), SequenceClassifierOutput(loss=tensor(1.2196, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[-0.0508,  0.2251, -0.0457],
        [-0.1034,  0.2494, -0.1685],
        [-0.1249,  0.2395,  0.0141],
        [-0.1715,  0.1827, -0.2325],
        [-0.2362,  0.3657,  0.0971],
        [ 0.0620,  0.1640, -0.3860]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)]
Printing last hidden states
tensor([[-0.3483,  0.2485,  0.1671, -0.0508,  0.2251, -0.0457],
        [-0.3406, -0.0271, -0.1022, -0.1034,  0.2494, -0.1685],
        

epoch:0, loss:-0.17791691422462463
<class 'dict'>
[SequenceClassifierOutput(loss=tensor(0.6328, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[2.9321, 1.1308, 1.6843],
        [3.1113, 1.2461, 1.5499],
        [3.3121, 1.1817, 1.3990],
        [2.9995, 1.4021, 1.5574],
        [2.8733, 1.2003, 1.7299],
        [3.1010, 1.1621, 1.5338]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None), SequenceClassifierOutput(loss=tensor(4.3609, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[-2.1508,  2.7776, -0.7291],
        [-1.9377,  2.7988, -0.6852],
        [-2.5241,  3.0473, -0.7228],
        [-2.2964,  2.6774, -0.5662],
        [-2.4452,  2.8751, -0.5770],
        [-2.4181,  2.7319, -0.6555]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)]
Printing last hidden states
tensor([[ 2.9321,  1.1308,  1.6843, -2.1508,  2.7776, -0.7291],
        [ 3.1113,  1.2461,  1.5499, -1.9377,  2.7988, -0.6852]

epoch:0, loss:-0.26167526841163635
<class 'dict'>
[SequenceClassifierOutput(loss=tensor(1.6597, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[4.2108, 1.5439, 2.4225],
        [4.2962, 1.6063, 2.4506],
        [4.1846, 1.4194, 2.6070],
        [4.1374, 1.4106, 2.4692],
        [4.1869, 1.5059, 2.4124],
        [4.1078, 1.5233, 2.4083]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None), SequenceClassifierOutput(loss=tensor(1.1659, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[-3.0037,  3.9510, -0.5614],
        [-3.1432,  3.7805, -0.6219],
        [-2.9730,  3.8102, -0.6911],
        [-2.8791,  3.8796, -0.5673],
        [-3.0711,  3.8137, -0.7747],
        [-3.1076,  3.8376, -0.7791]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)]
Printing last hidden states
tensor([[ 4.2108,  1.5439,  2.4225, -3.0037,  3.9510, -0.5614],
        [ 4.2962,  1.6063,  2.4506, -3.1432,  3.7805, -0.6219]

epoch:0, loss:-0.31249937415122986
<class 'dict'>
[SequenceClassifierOutput(loss=tensor(0.4625, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[4.7480, 1.8749, 2.7455],
        [4.7920, 1.8228, 2.5980],
        [4.7866, 1.8071, 2.9938],
        [4.8308, 1.7399, 2.7655],
        [4.8778, 1.7987, 2.8968],
        [4.9816, 1.7921, 2.6735]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None), SequenceClassifierOutput(loss=tensor(4.8112, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[-3.3566,  4.4263, -0.7442],
        [-3.2883,  4.3899, -0.6430],
        [-3.4284,  4.4104, -0.5559],
        [-3.3672,  4.4412, -0.5538],
        [-3.5931,  4.5528, -0.6479],
        [-3.1541,  4.3138, -0.8813]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)]
Printing last hidden states
tensor([[ 4.7480,  1.8749,  2.7455, -3.3566,  4.4263, -0.7442],
        [ 4.7920,  1.8228,  2.5980, -3.2883,  4.3899, -0.6430]

epoch:0, loss:-0.350986123085022
<class 'dict'>
[SequenceClassifierOutput(loss=tensor(1.5214, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[5.2720, 1.9372, 3.1063],
        [5.6215, 2.1153, 3.1005],
        [5.2766, 2.1109, 2.7818],
        [5.4172, 1.8002, 3.1404],
        [5.4382, 2.1813, 3.1332],
        [5.6060, 1.9655, 3.1076]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None), SequenceClassifierOutput(loss=tensor(8.7229, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[-3.8403,  4.8598, -1.0236],
        [-3.8026,  4.6085, -0.6693],
        [-3.7426,  5.0716, -0.7642],
        [-3.7528,  4.6362, -1.0890],
        [-4.0569,  5.0219, -0.8203],
        [-4.1454,  4.7768, -0.8135]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)]
Printing last hidden states
tensor([[ 5.2720,  1.9372,  3.1063, -3.8403,  4.8598, -1.0236],
        [ 5.6215,  2.1153,  3.1005, -3.8026,  4.6085, -0.6693],


...

epoch:1, loss:-7.078799724578857
<class 'dict'>
[SequenceClassifierOutput(loss=tensor(2.0957, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[29.1389, 25.0924, 25.4812],
        [29.9612, 25.3887, 25.8288],
        [29.4235, 24.6151, 25.5871],
        [29.9501, 25.1239, 25.8026],
        [29.0501, 24.8015, 24.6698],
        [28.3030, 24.5369, 24.8897]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None), SequenceClassifierOutput(loss=tensor(37.8535, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[-28.6461,  29.5862, -25.8756],
        [-27.4698,  28.3816, -24.7392],
        [-27.3086,  28.8581, -24.9542],
        [-27.5476,  28.9909, -25.3361],
        [-28.4639,  29.9058, -25.7755],
        [-28.7169,  29.8472, -26.0239]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)]
Printing last hidden states
tensor([[ 29.1389,  25.0924,  25.4812, -28.6461,  29.5862, -25.8756],
        [ 29.9

epoch:1, loss:-7.105643272399902
<class 'dict'>
[SequenceClassifierOutput(loss=tensor(2.1595, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[29.2380, 24.4481, 24.9965],
        [28.9519, 24.4928, 24.9252],
        [28.9652, 24.8390, 24.8576],
        [28.8047, 24.6266, 24.8083],
        [29.4326, 24.8706, 25.4465],
        [28.9974, 24.9736, 25.5488]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None), SequenceClassifierOutput(loss=tensor(37.8696, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[-28.3137,  29.6519, -25.7875],
        [-27.9180,  29.0865, -24.7624],
        [-28.2312,  29.4118, -25.4500],
        [-28.8607,  30.0371, -26.1793],
        [-28.8568,  29.7937, -25.9742],
        [-28.6203,  29.6455, -25.7726]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)]
Printing last hidden states
tensor([[ 29.2380,  24.4481,  24.9965, -28.3137,  29.6519, -25.7875],
        [ 28.9

epoch:1, loss:-7.0097737312316895
<class 'dict'>
[SequenceClassifierOutput(loss=tensor(3.5400, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[29.5678, 25.4506, 25.8476],
        [29.4981, 24.9818, 25.3344],
        [29.2939, 24.7423, 25.2374],
        [29.0944, 24.8791, 25.2120],
        [29.1667, 24.7923, 24.9321],
        [29.1830, 24.7473, 24.9332]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None), SequenceClassifierOutput(loss=tensor(38.8359, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[-28.4461,  29.6262, -25.5398],
        [-28.7506,  30.0912, -25.8797],
        [-28.9748,  30.2406, -26.1667],
        [-28.8056,  29.8653, -25.8321],
        [-27.9794,  29.4511, -25.3959],
        [-28.0513,  29.0221, -25.4265]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)]
Printing last hidden states
tensor([[ 29.5678,  25.4506,  25.8476, -28.4461,  29.6262, -25.5398],
        [ 29.

epoch:1, loss:-7.213237762451172
<class 'dict'>
[SequenceClassifierOutput(loss=tensor(2.7546, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[29.3492, 24.9581, 25.4175],
        [29.8264, 25.1416, 25.7614],
        [29.1119, 24.5910, 25.3049],
        [29.8289, 25.6082, 25.7583],
        [28.7590, 24.3599, 25.1665],
        [28.9254, 24.4041, 25.3075]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None), SequenceClassifierOutput(loss=tensor(58.2148, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[-28.3916,  29.4825, -25.5230],
        [-28.7848,  29.8467, -25.9438],
        [-28.7472,  29.8749, -26.3172],
        [-28.5388,  29.7117, -25.9667],
        [-28.3454,  29.6961, -25.7714],
        [-28.2814,  29.5874, -25.8925]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)]
Printing last hidden states
tensor([[ 29.3492,  24.9581,  25.4175, -28.3916,  29.4825, -25.5230],
        [ 29.8

epoch:1, loss:-7.25870418548584
<class 'dict'>
[SequenceClassifierOutput(loss=tensor(0.7402, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[29.6216, 24.4576, 25.5923],
        [29.8774, 25.6431, 26.0495],
        [29.2977, 24.9809, 25.5186],
        [29.7391, 25.3073, 26.1094],
        [29.4141, 25.1447, 25.4114],
        [29.2952, 25.0679, 25.9913]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None), SequenceClassifierOutput(loss=tensor(58.5700, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[-29.1873,  30.1541, -26.4645],
        [-28.8337,  29.6196, -26.0694],
        [-28.5826,  30.0485, -25.8523],
        [-28.0860,  28.9091, -24.8004],
        [-28.5168,  30.0130, -25.9532],
        [-29.3297,  30.1394, -26.3611]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)]
Printing last hidden states
tensor([[ 29.6216,  24.4576,  25.5923, -29.1873,  30.1541, -26.4645],
        [ 29.87

# Test Code

In [10]:
# standard pytorch way of doing things
# 1. create a custom Dataset 
# 2. pass the dataset to a dataloader
# 3. iterate the dataloader and pass the inputs to the model

max_len = 256
batch_size = 6
grad_step = 1
initialization_input = (max_len, batch_size)

#Reading datasets and initializing data loaders
dataset_location = '../2022.07.07_task5/'

#Gives us tweet_id, sentence, and label for each dataset.
train_data = read_task5(dataset_location , split = 'dev')
#test_data = read_task5(dataset_location , split = 'dev')#load test set
labels_to_ids = task5_labels_to_ids
#input_data = (train_data, dev_data, labels_to_ids)

dataloader_m1 = initialize_data(model1_tokenizer, initialization_input, train_data, labels_to_ids, shuffle = False)
dataloader_m2 = initialize_data(model2_tokenizer, initialization_input, train_data, labels_to_ids, shuffle = False)


complete_outputs, complete_label_ids = [], []

# iterate the QA and the AQ inputs simultaneously
for step, combined_batch in enumerate(zip(dataloader_m1, dataloader_m2)):
  # only forward pass so no dropout
  model.eval()
  batch_1, batch_2 = combined_batch


  # no back pass so no need to track variables for differentiation
  with torch.no_grad():
    inputs = {
        "input_ids": [batch_1[0].to(device, dtype = torch.long), batch_2[0].to(device, dtype = torch.long)],
        "attention_mask": [batch_1[1].to(device, dtype = torch.long), batch_2[1].to(device, dtype = torch.long)],
        "labels": [batch_1[2].to(device, dtype = torch.long), batch_2[2].to(device, dtype = torch.long)]
    }
    outputs = model(**inputs)
    tmp_eval_loss, logits = outputs[:2]
    logits = logits.detach().cpu().numpy()
    outputs = np.argmax(logits, axis=1)
    label_ids = inputs["labels"].detach().cpu().numpy()
  complete_outputs.extend(outputs)
  complete_label_ids.extend(label_ids)

print(complete_outputs, complete_label_ids)

NameError: name 'prepare_data' is not defined